In [1]:
import psycopg2

try:
    conn = psycopg2.connect("dbname='comments' user='admin' host='localhost' password=''")
    cursor = conn.cursor()
except:
    print("Failed to create connection")

In [2]:
delete_query = "DROP TABLE IF EXISTS comments;"
schema_query = "CREATE TABLE comments (comment_id SERIAL PRIMARY KEY, comment TEXT, date_created DATE, parent_id INT);"
insert_query = """
    INSERT INTO comments (comment_id, comment, date_created, parent_id) VALUES 
    (1, 'parent 1', '2021-01-01', NULL),
    (2, 'parent 2', '2021-01-01', NULL),
    (3, 'child of 1', '2021-01-02', 1),
    (4, 'another child of 1', '2021-01-03', 1),
    (5, 'child of 2', '2021-01-03', 2),
    (6, 'child of 3', '2021-01-03', 3),
    (7, 'child of 6', '2021-01-04', 6),
    (8, 'child of 7', '2021-01-05', 7),
    (9, 'child of 1', '2021-01-06', 1),
    (10, 'child of 5', '2021-01-05', 5);
    """
cursor.execute(delete_query)
cursor.execute(schema_query)
cursor.execute(insert_query)

In [3]:
select_basic = "SELECT * FROM comments"
cursor.execute(select_basic)
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'parent 1', datetime.date(2021, 1, 1), None)
(2, 'parent 2', datetime.date(2021, 1, 1), None)
(3, 'child of 1', datetime.date(2021, 1, 2), 1)
(4, 'another child of 1', datetime.date(2021, 1, 3), 1)
(5, 'child of 2', datetime.date(2021, 1, 3), 2)
(6, 'child of 3', datetime.date(2021, 1, 3), 3)
(7, 'child of 6', datetime.date(2021, 1, 4), 6)
(8, 'child of 7', datetime.date(2021, 1, 5), 7)
(9, 'child of 1', datetime.date(2021, 1, 6), 1)
(10, 'child of 5', datetime.date(2021, 1, 5), 5)


In [4]:
def run_select_query(cursor, query):
    cursor.execute(query)
    return cursor.fetchall()

"""
How this part would work would be: get all parents and display these. When
someone clicks to expand the parent, we fetch the children for each.
"""


all_parent_queries = """SELECT * FROM comments where parent_id IS NULL """
parents = run_select_query(cursor, all_parent_queries)
for row in parents:
    print(row)



# Gets all children for comment_id 1
cte_query = """

    WITH RECURSIVE child_comments AS (
        SELECT * FROM comments WHERE comment_id = 1
        UNION
        SELECT c.comment_id, c.comment, c.date_created, c.parent_id FROM comments c
        INNER JOIN child_comments cc ON c.parent_id = cc.comment_id
    ) SELECT * FROM child_comments ;
            """

rows = run_select_query(cursor, cte_query)

for row in rows:
    print(row)

# TODO: 
# - investigate cycles
# - investigate how to validate correct parent relationships
# - how do I deal with parent deletions


# Resources:
# https://www.postgresql.org/docs/9.1/queries-with.html
# https://www.citusdata.com/blog/2018/05/15/fun-with-sql-recursive-ctes/

(1, 'parent 1', datetime.date(2021, 1, 1), None)
(2, 'parent 2', datetime.date(2021, 1, 1), None)
(1, 'parent 1', datetime.date(2021, 1, 1), None)
(3, 'child of 1', datetime.date(2021, 1, 2), 1)
(4, 'another child of 1', datetime.date(2021, 1, 3), 1)
(9, 'child of 1', datetime.date(2021, 1, 6), 1)
(6, 'child of 3', datetime.date(2021, 1, 3), 3)
(7, 'child of 6', datetime.date(2021, 1, 4), 6)
(8, 'child of 7', datetime.date(2021, 1, 5), 7)


In [ ]:
# cycle in query
cycle_insert = "

In [20]:
cursor.close()